# Chapter 5. Test Doubles: Mocks, Fakes, and Stubs

## 5.1 Module Outline

(1) What is a test double?

(2) Different kinds of test double

(3) Why use test doubles?

(4) Using Monkeypatching to insert test doubles

## 5.2 What is a Test Double?

### 5.2.1 Test double

(1) Like "stunt doubles" who stand in for actors in films

(2) class under test doesn't know it isn't talking to the real object

(3) Allow you to control what happens to your class under test

### 5.2.2 Different kinds of test double

https://en.wikipedia.com/wiki/Test_Double

(1) Dummy object

(2) Test stub

(3) Test spy

(4) Mock object

(5) Fake object

### 5.2.3 Section outline

In an order of decreasing importance, we will look at

(1) Stub

(2) Fake

(3) Mock

(4) Test spy

(5) Dummy object

## 5.3 Example using a stub

A stub is any test double that has a very simple implementation with basically no logic or behavior.

### 5.3.1 Racing car example

In [ ]:
# SAVE AS sensor.py

import random

class Sensor(object):

    # The reading of the pressure value from the sensor is simulated in this implementation.
    # Because the focus of the exercise is on the other class.

    _OFFSET = 16
        
    def sample_pressure(self):
        pressure_telemetry_value = self.sample_actual_pressure()
        return Sensor._OFFSET + pressure_telemetry_value

    @staticmethod
    def sample_actual_pressure():
        # placeholder implementation that simulate a real sensor in a real tire
        pressure_telemetry_value = 6 * random.random() * random.random()
        return pressure_telemetry_value

In [ ]:
# SAVE AS alarm.py

from sensor import Sensor

class Alarm(object):
    def __init__(self, sensor=None):
        self._low_pressure_threshold = 17
        self._high_pressure_threshold = 21
        self._sensor = sensor or Sensor()
        self._is_alarm_on = False
        
    def check(self):
        psi_pressure_value = self._sensor.sample_pressure()
        if psi_pressure_value < self._low_pressure_threshold \
                or self._high_pressure_threshold < psi_pressure_value:
            self._is_alarm_on = True
            
    @property
    def is_alarm_on(self):
        return self._is_alarm_on

In [ ]:
# SAVE AS test_alarm.py

import unittest

from alarm import Alarm
from sensor import Sensor

class AlarmTest(unittest.TestCase):

    def test_alarm_is_off_by_default(self):
        alarm = Alarm()
        self.assertFalse(alarm.is_alarm_on)
        
    def test_check_too_low_pressure_sounds_alarm(self):
        alarm = Alarm(sensor=TestSensor(15))
        alarm.check()
        self.assertTrue(alarm.is_alarm_on)
        
    def test_check_too_high_pressure_sounds_alarm(self):
        alarm = Alarm(sensor=TestSensor(22))
        alarm.check()
        self.assertTrue(alarm.is_alarm_on)
        
    def test_check_normal_pressure_doesnt_sound_alarm(self):
        alarm = Alarm(sensor=TestSensor(18))
        alarm.check()
        self.assertFalse(alarm.is_alarm_on)
        
        
class TestSensor:
    
    def __init__(self, pressure):
        self._pressure = pressure
        
    def sample_pressure(self):
        return self._pressure

```bash
$ python -m unittest -v
test_alarm_is_off_by_default (test_alarm.AlarmTest) ... ok
test_check_normal_pressure_doesnt_sound_alarm (test_alarm.AlarmTest) ... ok
test_check_too_high_pressure_sounds_alarm (test_alarm.AlarmTest) ... ok
test_check_too_low_pressure_sounds_alarm (test_alarm.AlarmTest) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK
```

## 5.4 Using `unittest.Mock` to create a stub

(1) Use `unittest.Mock` instead of our hand-coded stub. The Mock framework will make sure that the mock object is actually substitutable for the collaborators it is replacing.

(2) Although the `unittest` framework mixes stub and mock together, they are actually different.

In [ ]:
# SAVE AS test_alarm.py

import unittest

from unittest.mock import Mock

from alarm import Alarm
from sensor import Sensor

class AlarmTest(unittest.TestCase):

    def test_alarm_is_off_by_default(self):
        alarm = Alarm()
        self.assertFalse(alarm.is_alarm_on)
        
    def test_check_too_low_pressure_sounds_alarm(self):
        alarm = Alarm(sensor=TestSensor(15))
        alarm.check()
        self.assertTrue(alarm.is_alarm_on)
        
    def test_check_too_high_pressure_sounds_alarm(self):
        alarm = Alarm(sensor=TestSensor(22))
        alarm.check()
        self.assertTrue(alarm.is_alarm_on)
        
    def test_check_normal_pressure_doesnt_sound_alarm(self):
        alarm = Alarm(sensor=TestSensor(18))
        alarm.check()
        self.assertFalse(alarm.is_alarm_on)
        
    def test_check_with_pressure_ok_with_mock_fw(self):
        test_sensor = Mock(Sensor)
        test_sensor.sample_pressure.return_value = 18
        alarm = Alarm(test_sensor)
        alarm.check()
        self.assertFalse(alarm.is_alarm_on)
        
        
class TestSensor:
    
    def __init__(self, pressure):
        self._pressure = pressure
        
    def sample_pressure(self):
        return self._pressure

```bash
$ python -m unittest -v
test_alarm_is_off_by_default (test_alarm.AlarmTest) ... ok
test_check_normal_pressure_doesnt_sound_alarm (test_alarm.AlarmTest) ... ok
test_check_too_high_pressure_sounds_alarm (test_alarm.AlarmTest) ... ok
test_check_too_low_pressure_sounds_alarm (test_alarm.AlarmTest) ... ok
test_check_with_pressure_ok_with_mock_fw (test_alarm.AlarmTest) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.007s

OK
```

## 5.5 Stub example summary

`Sensor.sample_pressure()` <== `TestSensor.sample_pressure()`

(1) Same interface

(2) Stub has no logic or advanced behavior

(3) A stub is not the same as a mock!

## 5.6 Example using a fake

Unlike a stub, a fack actually has some functionality, logic, an implementation that works at some level, although it is not suitable to be used in production.

Here we use a `FakeAccessWrapper` to avoid accessing the file system, i.e. replace `File` by `StringIO`. Other common fake usages are database and web server.

In [ ]:
# SAVE AS html_pages.py

import html as html_converter

class FileAccessWrapper:
    def __init__(self, filename):
        self.filename = filename
        
    def open(self):
        return open(self.filename, "r", encoding="UTF-8")

class HtmlPagesConverter:

    def __init__(self, file_access):
        """Read the file and note the positions of the page breaks so we can access them quickly"""
        self.file_access = file_access
        self.breaks = [0]
        with self.file_access.open() as f:
            while True:
                line = f.readline()
                if not line:
                    break
                line = line.rstrip()
                if "PAGE_BREAK" in line:
                    page_break_position = f.tell()
                    self.breaks.append(f.tell())
            self.breaks.append(f.tell())                

    def get_html_page(self, page):
        """Return html page with the given number (zero indexed)"""
        page_start = self.breaks[page]
        page_end = self.breaks[page+1]
        html = ""
        with self.file_access.open() as f:
            f.seek(page_start)
            while f.tell() != page_end:
                line = f.readline()
                line = line.rstrip()
                if "PAGE_BREAK" in line:
                    continue
                html += html_converter.escape(line, quote=True)
                html += "<br />"
        return html

In [ ]:
# SAVE AS test_html_pages.py

import unittest
import os
import tempfile
import io

from html_pages import HtmlPagesConverter, FileAccessWrapper

class HtmlPagesTest(unittest.TestCase):
    def test_inserts_br_tags_for_linebreaks(self):
        filename = os.path.join(tempfile.gettempdir(), "afile.txt")
        f = open(filename, "w", encoding="UTF-8")
        f.write("plain text\n")
        f.close()
        converter = HtmlPagesConverter(FileAccessWrapper(filename))
        new_text = converter.get_html_page(0)
        self.assertEqual("plain text<br />", new_text)
        
    def test_quotes_escaped(self):
        converter = HtmlPagesConverter(FakeFileWrapper("text with 'quotes'"))
        new_text = converter.get_html_page(0)
        self.assertEqual("text with &#x27;quotes&#x27;<br />", new_text)

    def test_random_access_pages(self):
        converter = HtmlPagesConverter(FakeFileWrapper("page one\nPAGE_BREAK\npage two\nPAGE_BREAK\npage three"))
        page_two = converter.get_html_page(1)
        self.assertEqual("page two<br />", page_two)
         
        
class FakeFileWrapper:
    def __init__(self, text):
        self.text = text
        
    def open(self):
        return io.StringIO(self.text)

In [ ]:
# SAVE AS test_html_pages.py

import unittest
import io

from html_pages import HtmlPagesConverter, FileAccessWrapper

class HtmlPagesTest(unittest.TestCase):
    def test_inserts_br_tags_for_linebreaks(self):
        # Replace the original file IO by StringIO.
        converter = HtmlPagesConverter(FakeFileWrapper("plain text\n"))
        new_text = converter.get_html_page(0)
        self.assertEqual("plain text<br />", new_text)
        
    def test_quotes_escaped(self):
        converter = HtmlPagesConverter(FakeFileWrapper("text with 'quotes'"))
        new_text = converter.get_html_page(0)
        self.assertEqual("text with &#x27;quotes&#x27;<br />", new_text)

    def test_random_access_pages(self):
        converter = HtmlPagesConverter(FakeFileWrapper("page one\nPAGE_BREAK\npage two\nPAGE_BREAK\npage three"))
        page_two = converter.get_html_page(1)
        self.assertEqual("page two<br />", page_two)
         
        
class FakeFileWrapper:
    def __init__(self, text):
        self.text = text
        
    def open(self):
        return io.StringIO(self.text)

```bash
$ python -m unittest -v
test_inserts_br_tags_for_linebreaks (test_html_pages.HtmlPagesTest) ... ok
test_quotes_escaped (test_html_pages.HtmlPagesTest) ... ok
test_random_access_pages (test_html_pages.HtmlPagesTest) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK
```

## 5.7 Example using a mock

### 5.7.1 Mock object

(1) The term "mock object" is overloaded and confusing. Lots of people refer to general test doubles as mock objects, while some people use stubs and mock objects interchangeably.

(2) Like stubs, mocks return hard-coded values and have little of their own behavior.

(3) Unlike stubs, mocks make assertions about what has happened in the test case and they may cause the test case to fail.

### 5.7.2 Three kinds of Assert.

In an order of increasing complexity:

(1) Check the return value or an exception.

(2) Check a state change (use a public API).

(3) Check a method call (use a mock or spy).

### 5.7.3 Interaction testing

Single sign-on service

In [ ]:
# SAVE AS single_sign_on.py

class FakeSingleSignOnRegistry:

    def __init__(self):
        self.tokens = set()

    def register(self, credentials):
        if are_valid(credentials):
            token = SSOToken()
            self.tokens.add(token)
            return token

    def is_valid(self, token):
        return token in self.tokens

    def end_session(self, token):
        self.tokens.remove(token)


class MockSingleSignOnRegistry:

    def __init__(self, expected_token, token_is_valid=True):
        self.expected_token = expected_token
        self.token_is_valid = token_is_valid
        self.is_valid_was_called = False

    def is_valid(self, token):
        self.is_valid_was_called = True
        if not token == self.expected_token:
            raise Exception("This mock was given an unexpected argument. Expected {0} got {1}".format(self.expected_token, token))
        return self.token_is_valid

    
class SSOToken:
    pass


def are_valid(credentials):
    #check the credentials
    return True

In [ ]:
# SAVE AS my_service.py

class MyService:

    def __init__(self, sso_registry):
        self.sso_registry = sso_registry

    def handle_request(self, request, token):
        if self.sso_registry.is_valid(token):
            return "hello world"
        return "please enter your login details"

In [ ]:
# SAVE AS test_my_service.py

import unittest

from my_service import MyService
from single_sign_on import *

class MyServiceTest(unittest.TestCase):
    def test_invalid_token(self):
        registry = FakeSingleSignOnRegistry()
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=None)
        self.assertIn("please enter your login details", response)
        
    def test_valid_token(self):
        registry = FakeSingleSignOnRegistry()
        token = registry.register("valid credentials")
        my_service = MyService(registry)
    
        response = my_service.handle_request("do stuff", token)
        self.assertIn("hello world", response)
        
    def test_invalid_token_with_mock(self):
        token = SSOToken()
        registry = MockSingleSignOnRegistry(expected_token=token, token_is_valid=False)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=token)
        self.assertTrue(registry.is_valid_was_called)

    def test_valid_token_with_mock(self):
        token = SSOToken()
        registry = MockSingleSignOnRegistry(expected_token=token, token_is_valid=True)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token)
        self.assertTrue(registry.is_valid_was_called)

```bash
$ python -m unittest -v
test_invalid_token (test_my_service.MyServiceTest) ... ok
test_invalid_token_with_mock (test_my_service.MyServiceTest) ... ok
test_valid_token (test_my_service.MyServiceTest) ... ok
test_valid_token_with_mock (test_my_service.MyServiceTest) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.000s

OK
```

## 5.8 Example using a spy

(1) A test spy is often used in a very similar way to a mock, to assert that particular interactions take place.

(2) A test spy is a type of test double that listens in on communication between the class you trying to test and the collaborating class. For our single sign-on service example, the test spy intercepts the call from class `MyService` to class `SingleSignOnRegistry`, and it may direct the call either to the real `SingleSignOnRegistry` or to the fake one.

(3) With a mock, it will blow up straight away, and fail the test, as soon as the wrong interaction happens; with a spy, it's just going to record what's going on, what calls have been made, and then in the assert part of the test, you ask it what happens and fail the test if the wrong thing happened.

In [ ]:
# SAVE AS single_sign_on.py

class FakeSingleSignOnRegistry:

    def __init__(self):
        self.tokens = set()

    def register(self, credentials):
        if are_valid(credentials):
            token = SSOToken()
            self.tokens.add(token)
            return token

    def is_valid(self, token):
        return token in self.tokens

    def end_session(self, token):
        self.tokens.remove(token)


class MockSingleSignOnRegistry:

    def __init__(self, expected_token, token_is_valid=True):
        self.expected_token = expected_token
        self.token_is_valid = token_is_valid
        self.is_valid_was_called = False

    def is_valid(self, token):
        self.is_valid_was_called = True
        if not token == self.expected_token:
            raise Exception("This mock was given an unexpected argument. Expected {0} got {1}".format(self.expected_token, token))
        return self.token_is_valid


class SpySingleSignOnRegistry:

    def __init__(self, accept_all_tokens = True):
        self.accept_all_tokens = accept_all_tokens
        self.checked_tokens = []

    def is_valid(self, token):
        self.checked_tokens.append(token)
        return self.accept_all_tokens


class SSOToken:
    pass


def are_valid(credentials):
    #check the credentials
    return True

In [ ]:
# SAVE AS my_service.py

class MyService:

    def __init__(self, sso_registry):
        self.sso_registry = sso_registry

    def handle_request_correctly(self, request, token):
        if self.sso_registry.is_valid(token):
            return "hello world"
        return "please enter your login details"
        
    def handle_request_wrong_token(self, request, token):
        if self.sso_registry.is_valid(None):
            return "hello world"
        return "please enter your login details"

    def handle_request_no_call_to_is_valid(self, request, token):
        if token:
            return "hello world"
        return "please enter your login details"

    handle_request = handle_request_correctly

In [ ]:
# SAVE AS test_my_service.py

import unittest

from my_service import MyService
from single_sign_on import *

class MyServiceTest(unittest.TestCase):
    def test_invalid_token(self):
        registry = FakeSingleSignOnRegistry()
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=None)
        self.assertIn("please enter your login details", response)
        
    def test_valid_token(self):
        registry = FakeSingleSignOnRegistry()
        token = registry.register("valid credentials")
        my_service = MyService(registry)
    
        response = my_service.handle_request("do stuff", token)
        self.assertIn("hello world", response)
        
    def test_invalid_token_with_mock(self):
        token = SSOToken()
        registry = MockSingleSignOnRegistry(expected_token=token, token_is_valid=False)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=token)
        self.assertTrue(registry.is_valid_was_called)

    def test_valid_token_with_mock(self):
        token = SSOToken()
        registry = MockSingleSignOnRegistry(expected_token=token, token_is_valid=True)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token)
        self.assertTrue(registry.is_valid_was_called)
        
    def test_invalid_token_with_spy(self):
        token = SSOToken()
        registry = SpySingleSignOnRegistry(accept_all_tokens=False)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=token)
        self.assertIn(token, registry.checked_tokens)

    def test_valid_token_with_spy(self):
        token = SSOToken()
        registry = SpySingleSignOnRegistry(accept_all_tokens=True)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token)
        self.assertIn(token, registry.checked_tokens)

* If `handle_request = handle_request_correctly`, 

```bash
$ python -m unittest -v
test_invalid_token (test_my_service.MyServiceTest) ... ok
test_invalid_token_with_mock (test_my_service.MyServiceTest) ... ok
test_invalid_token_with_spy (test_my_service.MyServiceTest) ... ok
test_valid_token (test_my_service.MyServiceTest) ... ok
test_valid_token_with_mock (test_my_service.MyServiceTest) ... ok
test_valid_token_with_spy (test_my_service.MyServiceTest) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.000s

OK
```

* If `handle_request = handle_request_wrong_token`, 

```bash
$ python -m unittest
.EFFEF
======================================================================
ERROR: test_invalid_token_with_mock (test_my_service.MyServiceTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/test_my_service.py", line 27, in test_invalid_token_with_mock
    response = my_service.handle_request("do stuff", token=token)
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/my_service.py", line 13, in handle_request_wrong_token
    if self.sso_registry.is_valid(None):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/single_sign_on.py", line 30, in is_valid
    raise Exception("This mock was given an unexpected argument. Expected {0} got {1}".format(self.expected_token, token))
Exception: This mock was given an unexpected argument. Expected <single_sign_on.SSOToken object at 0x7fc04fb0a2b0> got None

======================================================================
ERROR: test_valid_token_with_mock (test_my_service.MyServiceTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/test_my_service.py", line 35, in test_valid_token_with_mock
    response = my_service.handle_request("do stuff", token)
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/my_service.py", line 13, in handle_request_wrong_token
    if self.sso_registry.is_valid(None):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/single_sign_on.py", line 30, in is_valid
    raise Exception("This mock was given an unexpected argument. Expected {0} got {1}".format(self.expected_token, token))
Exception: This mock was given an unexpected argument. Expected <single_sign_on.SSOToken object at 0x7fc04fb0a2b0> got None

======================================================================
FAIL: test_invalid_token_with_spy (test_my_service.MyServiceTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/test_my_service.py", line 44, in test_invalid_token_with_spy
    self.assertIn(token, registry.checked_tokens)
AssertionError: <single_sign_on.SSOToken object at 0x7fc04fb0a320> not found in [None]

======================================================================
FAIL: test_valid_token (test_my_service.MyServiceTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/test_my_service.py", line 20, in test_valid_token
    self.assertIn("hello world", response)
AssertionError: 'hello world' not found in 'please enter your login details'

======================================================================
FAIL: test_valid_token_with_spy (test_my_service.MyServiceTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/test_my_service.py", line 52, in test_valid_token_with_spy
    self.assertIn(token, registry.checked_tokens)
AssertionError: <single_sign_on.SSOToken object at 0x7fc04fb0a320> not found in [None]

----------------------------------------------------------------------
Ran 6 tests in 0.001s

FAILED (failures=3, errors=2)
```

* If ``handle_request = handle_request_no_call_to_is_valid`,

```bash
$ python -m unittest
.FF.FF
======================================================================
FAIL: test_invalid_token_with_mock (test_my_service.MyServiceTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/test_my_service.py", line 28, in test_invalid_token_with_mock
    self.assertTrue(registry.is_valid_was_called)
AssertionError: False is not true

======================================================================
FAIL: test_invalid_token_with_spy (test_my_service.MyServiceTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/test_my_service.py", line 44, in test_invalid_token_with_spy
    self.assertIn(token, registry.checked_tokens)
AssertionError: <single_sign_on.SSOToken object at 0x7f8786d5d320> not found in []

======================================================================
FAIL: test_valid_token_with_mock (test_my_service.MyServiceTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/test_my_service.py", line 36, in test_valid_token_with_mock
    self.assertTrue(registry.is_valid_was_called)
AssertionError: False is not true

======================================================================
FAIL: test_valid_token_with_spy (test_my_service.MyServiceTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/mockspy_example_unittest/test_my_service.py", line 52, in test_valid_token_with_spy
    self.assertIn(token, registry.checked_tokens)
AssertionError: <single_sign_on.SSOToken object at 0x7f8786d5d2e8> not found in []

----------------------------------------------------------------------
Ran 6 tests in 0.001s

FAILED (failures=4)
```

## 5.9 Using `unittest.mock` to create a mock and a spy

`unittest.mock` is a spy framework primarily, and it's a little bit awkward to define mocks with it.

In [ ]:
# SAVE AS single_sign_on.py

class SingleSignOnRegistry:
    
    def register(self, credentials):
        pass
    
    def is_valid(self, token):
        pass
        
    def end_session(self, token):
        pass

class FakeSingleSignOnRegistry:

    def __init__(self):
        self.tokens = set()

    def register(self, credentials):
        if are_valid(credentials):
            token = SSOToken()
            self.tokens.add(token)
            return token

    def is_valid(self, token):
        return token in self.tokens

    def end_session(self, token):
        self.tokens.remove(token)


class MockSingleSignOnRegistry:

    def __init__(self, expected_token, token_is_valid=True):
        self.expected_token = expected_token
        self.token_is_valid = token_is_valid
        self.is_valid_was_called = False

    def is_valid(self, token):
        self.is_valid_was_called = True
        if not token == self.expected_token:
            raise Exception("This mock was given an unexpected argument. Expected {0} got {1}".format(self.expected_token, token))
        return self.token_is_valid


class SpySingleSignOnRegistry:

    def __init__(self, accept_all_tokens = True):
        self.accept_all_tokens = accept_all_tokens
        self.checked_tokens = []

    def is_valid(self, token):
        self.checked_tokens.append(token)
        return self.accept_all_tokens


class SSOToken:
    pass


def are_valid(credentials):
    #check the credentials
    return True

In [ ]:
# SAVE AS test_my_service.py

import unittest
from unittest.mock import *

from my_service import MyService
from single_sign_on import *

class MyServiceTest(unittest.TestCase):
    def test_invalid_token(self):
        registry = FakeSingleSignOnRegistry()
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=None)
        self.assertIn("please enter your login details", response)
        
    def test_valid_token(self):
        registry = FakeSingleSignOnRegistry()
        token = registry.register("valid credentials")
        my_service = MyService(registry)
    
        response = my_service.handle_request("do stuff", token)
        self.assertIn("hello world", response)
        
    def test_invalid_token_with_mock(self):
        token = SSOToken()
        registry = MockSingleSignOnRegistry(expected_token=token, token_is_valid=False)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=token)
        self.assertTrue(registry.is_valid_was_called)

    def test_valid_token_with_mock(self):
        token = SSOToken()
        registry = MockSingleSignOnRegistry(expected_token=token, token_is_valid=True)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token)
        self.assertTrue(registry.is_valid_was_called)
        
    def test_invalid_token_with_spy(self):
        token = SSOToken()
        registry = SpySingleSignOnRegistry(accept_all_tokens=False)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=token)
        self.assertIn(token, registry.checked_tokens)

    def test_valid_token_with_spy(self):
        token = SSOToken()
        registry = SpySingleSignOnRegistry(accept_all_tokens=True)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token)
        self.assertIn(token, registry.checked_tokens)
        
    def test_invalid_token_with_mocking_fw_as_spy(self):
        token = SSOToken()
        registry = Mock(SingleSignOnRegistry)
        registry.is_valid = Mock(return_value=False)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=token)
        registry.is_valid.assert_called_with(token)

    def test_valid_token_with_mocking_fw_as_spy(self):
        token = SSOToken()
        registry = Mock(SingleSignOnRegistry)
        registry.is_valid = Mock(return_value=True)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token)
        registry.is_valid.assert_called_with(token)

    def test_invalid_token_with_mocking_fw_as_mock(self):
        invalid_token = SSOToken()
        registry = Mock(SingleSignOnRegistry)
        def is_valid(token):
            if not token == invalid_token:
                raise Exception("Got the wrong token")
            return False
        registry.is_valid = Mock(side_effect=is_valid)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=invalid_token)
        registry.is_valid.assert_called_with(invalid_token)

    def test_valid_token_with_mocking_fw_as_mock(self):
        valid_token = SSOToken()
        registry = Mock(SingleSignOnRegistry)
        def is_valid(token):
            if not token == valid_token:
                raise Exception("Got the wrong token")
            return True
        registry.is_valid = Mock(side_effect=is_valid)
        my_service = MyService(registry)

        response = my_service.handle_request("do stuff", token=valid_token)
        registry.is_valid.assert_called_with(valid_token)

```bash
$ python -m unittest -v
test_invalid_token (test_my_service.MyServiceTest) ... ok
test_invalid_token_with_mock (test_my_service.MyServiceTest) ... ok
test_invalid_token_with_mocking_fw_as_mock (test_my_service.MyServiceTest) ... ok
test_invalid_token_with_mocking_fw_as_spy (test_my_service.MyServiceTest) ... ok
test_invalid_token_with_spy (test_my_service.MyServiceTest) ... ok
test_valid_token (test_my_service.MyServiceTest) ... ok
test_valid_token_with_mock (test_my_service.MyServiceTest) ... ok
test_valid_token_with_mocking_fw_as_mock (test_my_service.MyServiceTest) ... ok
test_valid_token_with_mocking_fw_as_spy (test_my_service.MyServiceTest) ... ok
test_valid_token_with_spy (test_my_service.MyServiceTest) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.007s

OK
```

## 5.10 Example using a dummy object

(1) Dummy object is often `None` or an empty collection.

(2) Use a dummy when you have a required argument that is not tested in this scenario.

If possible, you should try to change the dummy argument into optional.

In [ ]:
# SAVE AS fizzbuzz.py

def fizzbuzz(n, additional_rules):
    """     
    >>> fizzbuzz(2, None)
    '2'
    >>> fizzbuzz(3, None)
    'Fizz'
    >>> fizzbuzz(15, None)
    'FizzBuzz'
    >>> fizzbuzz(35, {7: "Whizz"})
    'BuzzWhizz'
    """
    answer = ""
    rules = {3: "Fizz", 5: "Buzz"}
    if additional_rules:
        rules.update(additional_rules)
    for divisor in sorted(rules.keys()):
        if n % divisor == 0:
            answer += rules[divisor]
    if not answer:
        answer = str(n)
    return answer

In [ ]:
# SAVE AS fizzbuzz.py

def fizzbuzz(n, additional_rules=None):
    """     
    >>> fizzbuzz(2)
    '2'
    >>> fizzbuzz(3)
    'Fizz'
    >>> fizzbuzz(15)
    'FizzBuzz'
    >>> fizzbuzz(35, {7: "Whizz"})
    'BuzzWhizz'
    """
    answer = ""
    rules = {3: "Fizz", 5: "Buzz"}
    if additional_rules:
        rules.update(additional_rules)
    for divisor in sorted(rules.keys()):
        if n % divisor == 0:
            answer += rules[divisor]
    if not answer:
        answer = str(n)
    return answer

## 5.11 Choosing to use a particular kind of test double

### 5.11.1 What's the difference?

(1) A Stub returns a hard-coded answer to any query.

(2) A Fake is a real but usually simpler implementation, yet unsuitable for production.

(3) A Mock is as a Stub, and additionally verifies interactions.

(4) A Test Spy lets you query afterwards to find out what happened.

(5) A Dummy is for when the interface requires an argument.

### 5.11.2 What is unit testing for?

(1) Understand what to build

(2) Document the units

(3) Design the units

(4) Regression protection

### 5.11.3 Isolation

(1) Replace a slow component by a Stub or Fake.

(2) Use a Mock to design a protocol.

(3) But test double may complicate the code and reduce the readability of the code.

## 5.12 Using Monkeypatching to insert a test double

### 5.12.1 Monkeypatching

Also called "metaprogramming".

(1) Changing code at runtime!

(2) Test the code without a good testability.

(3) Syntax: `with` context manager or `@patch` decorator

Before you resort to Monkeypatching, you should consider decoupling the code and improving the testability instead.

In [ ]:
# SAVE AS sensor.py

import random

class Sensor(object):

    # The reading of the pressure value from the sensor is simulated in this implementation.
    # Because the focus of the exercise is on the other class.

    _OFFSET = 16
        
    def sample_pressure(self):
        pressure_telemetry_value = self.sample_actual_pressure()
        return Sensor._OFFSET + pressure_telemetry_value

    @staticmethod
    def sample_actual_pressure():
        # placeholder implementation that simulate a real sensor in a real tire
        pressure_telemetry_value = 6 * random.random() * random.random()
        return pressure_telemetry_value

In [ ]:
# SAVE AS alarm.py

from sensor import Sensor

class Alarm(object):

    def __init__(self):
        self._low_pressure_threshold = 17
        self._high_pressure_threshold = 21
        self._sensor = Sensor()
        self._is_alarm_on = False

    def check(self):
        psi_pressure_value = self._sensor.sample_pressure()
        if psi_pressure_value < self._low_pressure_threshold or self._high_pressure_threshold < psi_pressure_value:
            self._is_alarm_on = True

    @property
    def is_alarm_on(self):
        return self._is_alarm_on

In [ ]:
# SAVE AS test_alarm.py

import unittest
from unittest.mock import *

from alarm import Alarm

class AlarmTest(unittest.TestCase):
    
    def test_check_with_too_high_pressure(self):
        with patch('alarm.Sensor') as test_sensor_class:
            test_sensor_instance = Mock()
            test_sensor_instance.sample_pressure.return_value = 22
            test_sensor_class.return_value=test_sensor_instance
            alarm = Alarm()
            alarm.check()
            self.assertTrue(alarm.is_alarm_on)

    @patch("alarm.Sensor")
    def test_check_with_too_low_pressure(self, test_sensor_class):
        test_sensor_instance = Mock()
        test_sensor_instance.sample_pressure.return_value = 15
        test_sensor_class.return_value = test_sensor_instance
        alarm = Alarm()
        alarm.check()
        self.assertTrue(alarm.is_alarm_on)

## 5.13 Module review

### 5.13.1 What is a Test Double?

(1) Stubs

(2) Fakes

(3) Mocks

(4) Spies

(5) Dummy Objects

### 5.13.2 Why use Test Doubles?

(1) Isolation

(2) Speed

(3) Design

### 5.13.3 Monkeypatching

To insert a test double